In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
raw = pd.read_csv('df.csv')

In [4]:
raw['LabelName'].unique()

array(['Bus', 'Truck'], dtype=object)

In [7]:
imagedf = raw[raw['ImageID']=='00006bdb1eb5cd74']

,XMin,YMin,XMax,YMax
1,0.276667,0.141604,0.697500,0.437343
2,0.702500,0.204261,0.999167,0.409774
